# 1. 确定性情况下模型搭建

In [7]:
from gurobipy import *

yield_info = [2.5,3,20]
plant_cost = [150,230,260]
sell_price =[170,150,36,10]
puchase_price = [238,210,10000]
require_info = [200,240,0]

# 定义模型
model = Model("Farming")

# 定义变量
x1 = model.addVar(name="x1", lb=0,vtype=GRB.CONTINUOUS)
x2 = model.addVar(name="x2", lb=0,vtype=GRB.CONTINUOUS)
x3 = model.addVar(name="x3", lb=0,vtype=GRB.CONTINUOUS)
y1 = model.addVar(name="y1", lb=0,vtype=GRB.CONTINUOUS)
y2 = model.addVar(name="y2", lb=0,vtype=GRB.CONTINUOUS)
y3 = model.addVar(name="y3", lb=0,vtype=GRB.CONTINUOUS)
w1 = model.addVar(name="w1", lb=0,vtype=GRB.CONTINUOUS)
w2 = model.addVar(name="w2", lb=0,vtype=GRB.CONTINUOUS)
w3 = model.addVar(name="w3", lb=0,vtype=GRB.CONTINUOUS)
w4 = model.addVar(name="w4", lb=0,vtype=GRB.CONTINUOUS)

# 定义目标函数
model.setObjective(
    x1*plant_cost[0]+x2*plant_cost[1]+x3*plant_cost[2]+y1*puchase_price[0]+y2*puchase_price[1]-w1*sell_price[0]-w2*sell_price[1]-w3*sell_price[2]-w4*sell_price[3],GRB.MINIMIZE
)

# 约束条件
model.addConstr(x1+x2+x3 <=500) # 土地限制
model.addConstr(yield_info[0]*x1+y1-w1>=require_info[0]) # Wheat需求限制
model.addConstr(yield_info[1]*x2+y2-w2>=require_info[1]) # Corn需求限制
model.addConstr(w3+w4<=yield_info[2]*x3) # 
model.addConstr(w3<=6000)

# 求解
model.optimize()

print('种植方案')
print('Wheat种植量：',x1.x,'Corn种植量：',x2.x,'Sugar Beets种植量：',x3.x)
print('购买方案')
print('Wheat购买量：',y1.x,'Corn购买量：',y2.x)
print('销售方案')
print('Wheat销售量：',w1.x,'Corn销售量：',w2.x,'Sugar Beets销售量(<=6000)：',w3.x,'Sugar Beets销售量(>6000)：',w4.x)
print(f"Obj: {model.objVal}")



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 10 columns and 13 nonzeros
Model fingerprint: 0xbe8c1566
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+03]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolved: 3 rows, 7 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.1200000e+33   2.000000e+30   5.120000e+03      0s
       4   -1.1860000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.186000000e+05
种植方案
Wheat种植量： 120.0 Corn种植量： 80.0 Sugar Beets种植量： 300.0
购买方案
Wheat购买量： 0.0 Corn购买量： 0.0
销售方案
Wheat销售量： 100.0 Corn销售量： 0.

# 2. 随机情况下模型搭建

现在假设我们可能存在三种情况，丰收(+20%)，正常，欠收(-20%)。那么这个情况下我们的策略是否需要改变呢？答案一定是需要改变的，下面分别先列出三种情况下的模型。

## 2.1 丰收情况下确定性模型结果

In [23]:
from gurobipy import *
import numpy as np
yield_info = np.array([2.5,3,20])*1.2
plant_cost = np.array([150,230,260])
sell_price =np.array([170,150,36,10])
puchase_price = np.array([238,210,10000])
require_info = np.array([200,240,0])

# 定义模型
model = Model("Farming")

# 定义变量
x1 = model.addVar(name="x1", lb=0,vtype=GRB.CONTINUOUS)
x2 = model.addVar(name="x2", lb=0,vtype=GRB.CONTINUOUS)
x3 = model.addVar(name="x3", lb=0,vtype=GRB.CONTINUOUS)
y1 = model.addVar(name="y1", lb=0,vtype=GRB.CONTINUOUS)
y2 = model.addVar(name="y2", lb=0,vtype=GRB.CONTINUOUS)
y3 = model.addVar(name="y3", lb=0,vtype=GRB.CONTINUOUS)
w1 = model.addVar(name="w1", lb=0,vtype=GRB.CONTINUOUS)
w2 = model.addVar(name="w2", lb=0,vtype=GRB.CONTINUOUS)
w3 = model.addVar(name="w3", lb=0,vtype=GRB.CONTINUOUS)
w4 = model.addVar(name="w4", lb=0,vtype=GRB.CONTINUOUS)

# 定义目标函数
model.setObjective(
    x1*plant_cost[0]+x2*plant_cost[1]+x3*plant_cost[2]+y1*puchase_price[0]+y2*puchase_price[1]-w1*sell_price[0]-w2*sell_price[1]-w3*sell_price[2]-w4*sell_price[3],GRB.MINIMIZE
)

# 约束条件
model.addConstr(x1+x2+x3 <=500) # 土地限制
model.addConstr(yield_info[0]*x1+y1-w1>=require_info[0]) # Wheat需求限制
model.addConstr(yield_info[1]*x2+y2-w2>=require_info[1]) # Corn需求限制
model.addConstr(w3+w4<=yield_info[2]*x3) # 
model.addConstr(w3<=6000)

# 求解
model.optimize()

print('种植方案')
print('Wheat种植量：',x1.x,'Corn种植量：',x2.x,'Sugar Beets种植量：',x3.x)
print('购买方案')
print('Wheat购买量：',y1.x,'Corn购买量：',y2.x)
print('销售方案')
print('Wheat销售量：',w1.x,'Corn销售量：',w2.x,'Sugar Beets销售量(<=6000)：',w3.x,'Sugar Beets销售量(>6000)：',w4.x)
print(f"Obj: {model.objVal}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 10 columns and 16 nonzeros
Model fingerprint: 0x236984d8
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 6e+03]
Presolve removed 8 rows and 10 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5429350e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.542935000e+05
种植方案
Wheat种植量： 135.83 Corn种植量： 80.57 Sugar Beets种植量： 279.1
购买方案
Wheat购买量： 0.0 Corn购买量： 0.0
销售方案
Wheat销售量： 207.49 Corn销售量： 50.051999999999964 Sugar Beets销售量(<=6000)： 6000.0 Sugar Beets销售量(

## 2.2 欠收情况下确定性模型结果

In [24]:
from gurobipy import *
import numpy as np
yield_info = np.array([2.5,3,20])*0.8
plant_cost = np.array([150,230,260])
sell_price =np.array([170,150,36,10])
puchase_price = np.array([238,210,10000])
require_info = np.array([200,240,0])

# 定义模型
model = Model("Farming")

# 定义变量
x1 = model.addVar(name="x1", lb=0,vtype=GRB.CONTINUOUS)
x2 = model.addVar(name="x2", lb=0,vtype=GRB.CONTINUOUS)
x3 = model.addVar(name="x3", lb=0,vtype=GRB.CONTINUOUS)
y1 = model.addVar(name="y1", lb=0,vtype=GRB.CONTINUOUS)
y2 = model.addVar(name="y2", lb=0,vtype=GRB.CONTINUOUS)
y3 = model.addVar(name="y3", lb=0,vtype=GRB.CONTINUOUS)
w1 = model.addVar(name="w1", lb=0,vtype=GRB.CONTINUOUS)
w2 = model.addVar(name="w2", lb=0,vtype=GRB.CONTINUOUS)
w3 = model.addVar(name="w3", lb=0,vtype=GRB.CONTINUOUS)
w4 = model.addVar(name="w4", lb=0,vtype=GRB.CONTINUOUS)

# 定义目标函数
model.setObjective(
    x1*plant_cost[0]+x2*plant_cost[1]+x3*plant_cost[2]+y1*puchase_price[0]+y2*puchase_price[1]-w1*sell_price[0]-w2*sell_price[1]-w3*sell_price[2]-w4*sell_price[3],GRB.MINIMIZE
)

# 约束条件
model.addConstr(x1+x2+x3 <=500) # 土地限制
model.addConstr(yield_info[0]*x1+y1-w1>=require_info[0]) # Wheat需求限制
model.addConstr(yield_info[1]*x2+y2-w2>=require_info[1]) # Corn需求限制
model.addConstr(w3+w4<=yield_info[2]*x3) # 
model.addConstr(w3<=6000)

# 求解
model.optimize()

print('种植方案')
print('Wheat种植量：',x1.x,'Corn种植量：',x2.x,'Sugar Beets种植量：',x3.x)
print('购买方案')
print('Wheat购买量：',y1.x,'Corn购买量：',y2.x)
print('销售方案')
print('Wheat销售量：',w1.x,'Corn销售量：',w2.x,'Sugar Beets销售量(<=6000)：',w3.x,'Sugar Beets销售量(>6000)：',w4.x)
print(f"Obj: {model.objVal}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 10 columns and 16 nonzeros
Model fingerprint: 0xed6dde1f
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 6e+03]
Presolve removed 8 rows and 10 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.1679480e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective -5.167948000e+04
种植方案
Wheat种植量： 135.83 Corn种植量： 80.57 Sugar Beets种植量： 279.1
购买方案
Wheat购买量： 0.0 Corn购买量： 46.63199999999998
销售方案
Wheat销售量： 71.66000000000003 Corn销售量： 0.0 Sugar Beets销售量(<=6000)： 4465.6 Sugar

## 2.3 随机情况下模型搭建


In [26]:
from gurobipy import *
import numpy as np
normal_yield_info = np.array([2.5,3,20])
up_yield_info = np.array([2.5,3,20])*1.2
down_yield_info = np.array([2.5,3,20])*0.8
plant_cost = np.array([150,230,260])
sell_price =np.array([170,150,36,10])
puchase_price = np.array([238,210,10000])
require_info = np.array([200,240,0])

# 定义模型
model = Model("Farming")
C = range(3)
# 定义变量
x1 = model.addVar(name="x1", lb=0,vtype=GRB.CONTINUOUS)
x2 = model.addVar(name="x2", lb=0,vtype=GRB.CONTINUOUS)
x3 = model.addVar(name="x3", lb=0,vtype=GRB.CONTINUOUS)
y1 = model.addVars(C,name="y1", lb=0,vtype=GRB.CONTINUOUS)
y2 = model.addVars(C,name="y2", lb=0,vtype=GRB.CONTINUOUS)
y3 = model.addVars(C,name="y3", lb=0,vtype=GRB.CONTINUOUS)
w1 = model.addVars(C,name="w1", lb=0,vtype=GRB.CONTINUOUS)
w2 = model.addVars(C,name="w2", lb=0,vtype=GRB.CONTINUOUS)
w3 = model.addVars(C,name="w3", lb=0,vtype=GRB.CONTINUOUS)
w4 = model.addVars(C,name="w4", lb=0,vtype=GRB.CONTINUOUS)

# 定义目标函数
obj = LinExpr()
obj += x1*plant_cost[0]+x2*plant_cost[1]+x3*plant_cost[2]
obj += 1/3*(y1[0]*puchase_price[0]+y2[0]*puchase_price[1]-w1[0]*sell_price[0]-w2[0]*sell_price[1]-w3[0]*sell_price[2]-w4[0]*sell_price[3])
obj += 1/3*(y1[1]*puchase_price[0]+y2[1]*puchase_price[1]-w1[1]*sell_price[0]-w2[1]*sell_price[1]-w3[1]*sell_price[2]-w4[1]*sell_price[3])
obj += 1/3*(y1[2]*puchase_price[0]+y2[2]*puchase_price[1]-w1[2]*sell_price[0]-w2[2]*sell_price[1]-w3[2]*sell_price[2]-w4[2]*sell_price[3])

model.setObjective(
    obj,GRB.MINIMIZE
)

# 约束条件
model.addConstr(x1+x2+x3 <=500) # 土地限制

# 正常情况下
model.addConstr(normal_yield_info[0]*x1+y1[0]-w1[0]>=require_info[0]) # Wheat需求限制
model.addConstr(normal_yield_info[1]*x2+y2[0]-w2[0]>=require_info[1]) # Corn需求限制
model.addConstr(w3[0]+w4[0]<=normal_yield_info[2]*x3) 

# 丰收情况下
model.addConstr(up_yield_info[0]*x1+y1[1]-w1[1]>=require_info[0]) # Wheat需求限制
model.addConstr(up_yield_info[1]*x2+y2[1]-w2[1]>=require_info[1]) # Corn需求限制
model.addConstr(w3[1]+w4[1]<=up_yield_info[2]*x3) 

# 欠收情况下
model.addConstr(down_yield_info[0]*x1+y1[2]-w1[2]>=require_info[0]) # Wheat需求限制
model.addConstr(down_yield_info[1]*x2+y2[2]-w2[2]>=require_info[1]) # Corn需求限制
model.addConstr(w3[2]+w4[2]<=down_yield_info[2]*x3) 

# 生产限制.
model.addConstrs(w3[i]<=6000 for i in C)

# 求解
model.optimize()
print('='*50)
print('种植方案')
print(f"Wheat种植量：{x1.x}, Corn种植量：{x2.x}, Sugar Beets种植量：{x3.x}")
print('正常情况下')
print(f"Wheat购买量：{y1[0].x}, Corn购买量：{y2[0].x}")
print(f"Wheat销售量：{w1[0].x}, Corn销售量：{w2[0].x}, Sugar Beets销售量(<=6000)：{w3[0].x}, Sugar Beets销售量(>6000)：{w4[0].x}")
print('丰收情况下')
print(f"Wheat购买量：{y1[1].x}, Corn购买量：{y2[1].x}")
print(f"Wheat销售量：{w1[1].x}, Corn销售量：{w2[1].x}, Sugar Beets销售量(<=6000)：{w3[1].x}, Sugar Beets销售量(>6000)：{w4[1].x}")
print('欠收情况下')
print(f"Wheat购买量：{y1[2].x}, Corn购买量：{y2[2].x}")
print(f"Wheat销售量：{w1[2].x}, Corn销售量：{w2[2].x}, Sugar Beets销售量(<=6000)：{w3[2].x}, Sugar Beets销售量(>6000)：{w4[2].x}")
print(f"Obj: {model.objVal}")
print('='*50)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 24 columns and 33 nonzeros
Model fingerprint: 0x1e2e01f5
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+03]
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.5600000e+33   6.000000e+30   2.560000e+03      0s
      13   -1.0839000e+05   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.083900000e+05
种植方案
Wheat种植量：170.0, Corn种植量：80.0, Sugar Beets种植量：250.0
正常情况下
Wheat购买量：0.0, Corn购买量：0.0
Wheat销售量：225.0, Corn销售量：0.0, 

## EVPI 和 VSS引入

如果我们现在不知道具体的情况，按平均概率来看这个事件，我们最优决策下的目标函数为：108390

如果我们现在有一个`Perfect Information`，也就是我们知道所有信息，那么我们最优决策下的目标函数为：(59950+167667+118600)/3=115404.66

**EVPI**:两者之间存在一个差值，这个差值就是`信息带来的价值` = 115404-108390=7014



In [4]:
150*240

36000

In [8]:
150*3.6

540.0

In [11]:
50400-60*240

36000